# Import  Library 

In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time 
import pandas as pd 
from itertools import permutations


# Get Country Code 

In [2]:
browser = webdriver.Firefox()
browser.get('https://www.trademap.org/Bilateral_TS.aspx?nvpm=1|360||156||TOTAL|||2|1|1|1|2|1|1|1||1')
html_source = browser.page_source
browser.quit()

soup = BeautifulSoup(html_source,'html.parser')
#class "postText" is not defined in the source code
comments = soup.findAll('option')

kode = []
dct= {}
for option in soup.find_all('option'):
    dct[option.text] = option['value']
    kode.append(option['value'])
kode = kode[113:366]

kode_negara = {}
for  key, value in dct.items():
    if value in  kode:
        kode_negara[key] = value
        
 

In [3]:
kombinasi_negara = []
for  key, value in kode_negara.items():
    kombinasi_negara.append(('360', value))
    


In [4]:
for val, i in enumerate(kombinasi_negara):
    if i[0] == i[1]:
        print(val)

110


In [5]:
kombinasi_negara.pop(110)

('360', '360')

In [6]:
kombinasi_negara[1]

('360', '577')

# Get HTML  Importer  Data

In [7]:
def extract_import(negara, partner):

    browser = webdriver.Firefox()
    browser.get(f"https://www.trademap.org/Bilateral_TS.aspx?nvpm=1|{negara}||{partner}||TOTAL|||2|1|1|1|2|1|1|1|1|1")
    time.sleep(3)
    
    select = Select(browser.find_element_by_id('ctl00_NavigationControl_DropDownList_ProductClusterLevel'))
    select.select_by_value('4')

    time.sleep(3)
    select = Select(browser.find_element_by_id('ctl00_PageContent_GridViewPanelControl_DropDownList_NumTimePeriod'))
    select.select_by_value('5')

    time.sleep(3)
    select = Select(browser.find_element_by_id('ctl00_PageContent_GridViewPanelControl_DropDownList_PageSize'))
    select.select_by_value('300')
    time.sleep(3)
    html_source = []
    jumlah_page = 0
    html_source.append((browser.page_source))
    jumlah_page +=1 
    print (f" Start Page {jumlah_page}...")
    
    while True :
        try :
            WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, "//table[@id='ctl00_PageContent_MyGridView1']//table/tbody/tr//td/span//following::td[1]/a"))).click()
            html_source.append((browser.page_source))
            jumlah_page +=1 
            print (f" Start Page {jumlah_page}...")
        except TimeoutException:
            print("Finished \n")
            break

        
    
   


    browser.quit()

    

    return html_source





# Get HTML Export Data

In [8]:
def extract_export(negara, partner ):

    browser = webdriver.Firefox()
    browser.get(f"https://www.trademap.org/Bilateral_TS.aspx?nvpm=1|{negara}||{partner}||TOTAL|||2|1|1|2|2|1|1|1|1|1")
    time.sleep(3)
    
    select = Select(browser.find_element_by_id('ctl00_NavigationControl_DropDownList_ProductClusterLevel'))
    select.select_by_value('4')

    time.sleep(3)
    select = Select(browser.find_element_by_id('ctl00_PageContent_GridViewPanelControl_DropDownList_NumTimePeriod'))
    select.select_by_value('5')

    time.sleep(3)
    select = Select(browser.find_element_by_id('ctl00_PageContent_GridViewPanelControl_DropDownList_PageSize'))
    select.select_by_value('300')
    time.sleep(3)
    html_source = []
    jumlah_page = 0
    html_source.append((browser.page_source))
    jumlah_page +=1 
    print (f" Start Page {jumlah_page}...")
    
    while True :
        try :
            WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, "//table[@id='ctl00_PageContent_MyGridView1']//table/tbody/tr//td/span//following::td[1]/a"))).click()
            html_source.append((browser.page_source))
            jumlah_page +=1 
            print (f" Start Page {jumlah_page}...")
        except TimeoutException:
            print("Finished \n")
            break

        
    
   


    browser.quit()

    

    return html_source






# Extract  Data

In [9]:
def data_import_export_extract(html_source):
    soup = BeautifulSoup(html_source,'html.parser')

    data = soup.findAll('td',{'class':['tabContent']})
    
    web = []
    for item in data:
        tdTags = item.find_all("tr")
    for item in tdTags:
        asi = item.find_all("td")
        for asii in asi :
                #print(no , asii.text)
                #product_code.append(asii.text.strip())
            web.append(asii.text.strip())
    web = web[24:]
    
    return web


In [10]:
def insert_into_list(web):
    m = len(web)
    n =20

    product_code =[]
    product_label = []
    value_2016 =[]
    value_2017 =[]
    value_2018 =[]
    value_2019=[]
    value_2020=[]

    for i in range (11, m, n ):
        #print(web[i])
        product_code.append(web[i])
        product_label.append(web[i+1])
        value_2016.append(web[i+2])
        value_2017.append(web[i+3])
        value_2018.append(web[i+4])
        value_2019.append(web[i+5])
        value_2020.append(web[i+6])
    
    # make dataframe
    kolom = ['product_code', 'product_label','2016','2017','2018','2019','2020']
    df = pd.DataFrame(columns= kolom)
    df['product_code'] = product_code
    df['product_label'] = product_label
    df['2016'] = value_2016
    df['2017'] = value_2017
    df['2018'] = value_2018
    df['2019'] = value_2019
    df['2020'] = value_2020
    return df

In [11]:
def get_key(val):
    for key, value in kode_negara.items():
         if val == value:
            return key

# For Data Importir 

In [12]:
kolom = ['product_code', 'product_label','2016','2017','2018','2019','2020']
data_importir = pd.DataFrame(columns= kolom)
no =1 
for i,kombinasi in enumerate(kombinasi_negara,1):
    if no <= 2:
        print(f"{i}/{len(kombinasi_negara)} Started")
        negara = kombinasi[0]
        partner = kombinasi[1]
        nama_negara = get_key(negara)
        nama_partner = get_key(partner)
        
        data_import = extract_import(negara=negara, partner= partner )
        for data in data_import:
            web = data_import_export_extract(data)
            df = insert_into_list(web)
            df['importer'] = nama_negara  
            df['exporter'] = nama_partner
            data_importir = pd.concat([data_importir, df], ignore_index= True)
            data_importir = data_importir[data_importir.product_code != "1" ] 
            data_importir = data_importir.reset_index(drop=True)        
        no += 1
    else :
        break

    
data_importir


1/254 Started
 Start Page 1...
 Start Page 2...
 Start Page 3...
 Start Page 4...
 Start Page 5...
Finished 

2/254 Started
 Start Page 1...
 Start Page 2...
 Start Page 3...
 Start Page 4...
 Start Page 5...
Finished 



,product_code,product_label,2016,2017,2018,2019,2020,importer,exporter
0,8213,"Scissors, tailors' shears and similar shears, ...",0,0,0,0,0,Indonesia,Afghanistan
1,8415,Air conditioning machines comprising a motor-d...,0,0,0,0,0,Indonesia,Afghanistan
2,8423,"Weighing machinery, incl. weight-operated coun...",0,0,0,0,0,Indonesia,Afghanistan
3,8425,Pulley tackle and hoists (other than skip hois...,0,0,0,0,0,Indonesia,Afghanistan
4,8440,"Bookbinding machinery, incl. book-sewing machi...",0,0,0,0,0,Indonesia,Afghanistan
...,...,...,...,...,...,...,...,...,...
2437,9507,"Fishing rods, fish-hooks and other line fishin...",0,0,0,0,0,Indonesia,Africa not elsewhere specified
2438,9609,"Pencils, crayons, pencil leads, pastels, drawi...",0,0,0,0,0,Indonesia,Africa not elsewhere specified
2439,9615,"Combs, hair-slides and the like; hairpins; cur...",0,0,0,0,0,Indonesia,Africa not elsewhere specified
2440,9617,"Vacuum flasks and other vacuum vessels, and pa...",0,0,0,0,0,Indonesia,Africa not elsewhere specified


# For Data Exportir

In [13]:
kolom = ['product_code', 'product_label','2016','2017','2018','2019','2020']
data_exportir = pd.DataFrame(columns= kolom)
no = 1 
for i,kombinasi in enumerate(kombinasi_negara,1):
    if no <= 2:
        print(f"{i}/{len(kombinasi_negara)} Started")
        negara = kombinasi[0]
        partner = kombinasi[1]
        nama_negara = get_key(negara)
        nama_partner = get_key(partner)
        data_export = extract_export(negara=negara, partner= partner)
        for data in data_export:
            web = data_import_export_extract(data)
            df = insert_into_list(web)
            df['importer'] = nama_partner  
            df['exporter'] = nama_negara
            data_exportir = pd.concat([data_exportir, df], ignore_index= True)
            data_exportir = data_exportir[data_exportir.product_code != "1" ] 
            data_exportir = data_exportir.reset_index(drop=True) 
        no+=1
    else:
        break


data_exportir

1/254 Started
 Start Page 1...
 Start Page 2...
 Start Page 3...
 Start Page 4...
 Start Page 5...
Finished 

2/254 Started
 Start Page 1...
 Start Page 2...
 Start Page 3...
 Start Page 4...
 Start Page 5...
Finished 



,product_code,product_label,2016,2017,2018,2019,2020,importer,exporter
0,1806,Chocolate and other food preparations containi...,0,0,0,0,0,Afghanistan,Indonesia
1,2006,"Vegetables, fruit, nuts, fruit-peel and other ...",0,0,0,0,0,Afghanistan,Indonesia
2,0105,"Live poultry, ""fowls of the species Gallus dom...",0,0,0,0,0,Afghanistan,Indonesia
3,0201,"Meat of bovine animals, fresh or chilled",0,0,0,0,0,Afghanistan,Indonesia
4,0203,"Meat of swine, fresh, chilled or frozen",0,0,0,0,0,Afghanistan,Indonesia
...,...,...,...,...,...,...,...,...,...
2437,9507,"Fishing rods, fish-hooks and other line fishin...",0,0,0,0,0,Africa not elsewhere specified,Indonesia
2438,9609,"Pencils, crayons, pencil leads, pastels, drawi...",0,0,0,0,0,Africa not elsewhere specified,Indonesia
2439,9615,"Combs, hair-slides and the like; hairpins; cur...",0,0,0,0,0,Africa not elsewhere specified,Indonesia
2440,9617,"Vacuum flasks and other vacuum vessels, and pa...",0,0,0,0,0,Africa not elsewhere specified,Indonesia
